In [1]:
import pandas as pd
import sqlite3
import requests
import numpy as np
import folium
import folium.plugins 
import matplotlib.pyplot as plt
from IPython.display import clear_output

# Data for the RV grouping project

Online I found a complete list of camp grounds in CSV format from http://www.poi-factory.com/node/17657

I will now extract all the required information
- Latitude and Longitude Data
- Name of the Camp Site
- Save the data to a sql3 database for wasy retreival and querying

In [2]:
df_raw = pd.read_csv('ParksandCampsites.csv', header=None)

In [3]:
df_raw = df_raw.set_axis(['Longitude', 'Latitude', 'Campsite_Name','Address'], axis=1)
df_raw

,Longitude,Latitude,Campsite_Name,Address
0,-127.679933,55.249428,'KSAN Historical Village Campground,"1450 River Rd,Haselton BC ,250 842-5940"
1,-73.403468,44.030277,10 Acres Campground and RV Park,"9 10 Acre Dr,Addison,*Vergennes VT *05491 ,802..."
2,-75.997158,44.276788,1000 Islands Campground,"42099 SR 12,Fishers Landing NY *13641 ,315 686..."
3,-76.054170,44.350050,1000 Islands Camping Resort,"382 1000 Islands Pkwy,Lansdowne,*South Leeds a..."
4,-76.216943,43.891877,1000 Islands/Association Island K O A,"15530 Snowshoe Rd,Henderson NY *13650 ,315 955..."
...,...,...,...,...
17152,-94.849806,48.847923,Zippel Bay SP,"54th Ave NW,Williams MN ,218 783-6252"
17153,-79.850677,35.603549,Zooland Family Campground,"3671 Pisgah Covered Bridge Rd,Asheboro NC ,336..."
17154,-92.122085,44.301126,Zumbro Bottoms Horse Campground,"T-70,Kellog MN ,651 345-3401"
17155,-114.014515,35.224768,Zuni Village RV Park,"2840 Airway Ave,Kingman AZ ,928 692-6202"


In [4]:
con = sqlite3.connect('Campsite_data_4k-all.db')
try:
    df_raw.to_sql('Sites',con)
except:
    pass
con.close()

In [5]:
con = sqlite3.connect('Campsite_data_4k-all.db')
query = ''' SELECT Campsite_Name, Latitude, Longitude FROM Sites WHERE Campsite_Name not like '%closed%'   '''
df_RV = pd.read_sql_query(query,con).set_index('Campsite_Name').drop_duplicates()
con.close
df_RV


,Latitude,Longitude
Campsite_Name,,
'KSAN Historical Village Campground,55.249428,-127.679933
10 Acres Campground and RV Park,44.030277,-73.403468
1000 Islands Campground,44.276788,-75.997158
1000 Islands Camping Resort,44.350050,-76.054170
1000 Islands/Association Island K O A,43.891877,-76.216943
...,...,...
Zippel Bay SP,48.847923,-94.849806
Zooland Family Campground,35.603549,-79.850677
Zumbro Bottoms Horse Campground,44.301126,-92.122085


## Map with the location Data

In [ ]:
CLIENT_ID = 'XX' # your Foursquare ID
CLIENT_SECRET = 'XX' # your Foursquare Secret
ACCESS_TOKEN = 'XX' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 60
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

## Neighboring venues data

Now I will use Foursquare to download neighboring venue data and save them to my sql3 database

In [7]:
try:
    con = sqlite3.connect('Campsite_data_4k-all.db')
    query = ''' SELECT distinct(Campsite_Name) FROM Venues'''
    completed_venues = pd.read_sql_query(query,con)['Campsite_Name'].tolist()
    con.close
except:
    completed_venues = []

num_iter = 0
print('Number of completed venues that will not be run again: {}'.format(len(completed_venues)))
for key, item in df_RV.reset_index().iterrows():
    clear_output(wait=True)
    venues_list = []
    num_iter = num_iter +1
    
    # create the API request URL
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        item['Latitude'], 
        item['Longitude'], 
        4000, #radius
        LIMIT)
    
    if not (item['Campsite_Name'] in completed_venues):
        print('Number: {}, New Site: {}'.format(num_iter,item['Campsite_Name']))
        try:
            results = requests.get(url).json()["response"]['groups'][0]['items']
            for v in results:
                temp = {}
                temp['Campsite_Name']= item['Campsite_Name']
                temp['Latitude']= item['Latitude']
                temp['Longitude']= item['Longitude']
                temp['Venue']=v['venue']['name']
                temp['Venue_Latitude']=v['venue']['location']['lat']
                temp['Venue_Longitude']=v['venue']['location']['lng']  
                temp['Venue_Category']=v['venue']['categories'][0]['name']
                venues_list.append(temp)

            nearby_venues = pd.DataFrame(venues_list)
            con = sqlite3.connect('Campsite_data_4k-all.db')
            nearby_venues.to_sql('Venues',con,if_exists='append')
            con.close()
        except:
            print('Failed')

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



## Check data integrity

In [8]:
con = sqlite3.connect('Campsite_data_4k-all.db')
query = ''' SELECT * FROM Venues'''
completed_venues = pd.read_sql_query(query,con).drop('index',axis=1)
completed_venues

,Campsite_Name,Latitude,Longitude,Venue,Venue_Latitude,Venue_Longitude,Venue_Category
0,'KSAN Historical Village Campground,55.249428,-127.679933,Ksan Historical Village,55.250044,-127.677005,Gift Shop
1,'KSAN Historical Village Campground,55.249428,-127.679933,'Ksan Historic Village and Museum,55.251827,-127.674639,Museum
2,'KSAN Historical Village Campground,55.249428,-127.679933,Polly's Cafe,55.255564,-127.675666,Café
3,'KSAN Historical Village Campground,55.249428,-127.679933,Polly Sargent Park,55.256735,-127.676958,Park
4,'KSAN Historical Village Campground,55.249428,-127.679933,Gitanmaax Market,55.257602,-127.662515,Market
...,...,...,...,...,...,...,...
146383,Travelers N Park and Campground,31.751306,-98.921703,Jerrys Burgers N Shakes,31.744784,-98.940153,American Restaurant
146384,Travelers N Park and Campground,31.751306,-98.921703,Burger King,31.737710,-98.957946,Fast Food Restaurant
146385,Travelers N Park and Campground,31.751306,-98.921703,ACE DISPOSAL,31.780158,-98.913099,Business Service
146386,Travelers N Park and Campground,31.751306,-98.921703,Happy China Buffet,31.736954,-98.954941,Chinese Restaurant
